In [18]:
import torch

import composer
from composer.datasets import coco_mmdet
from composer.models import composer_yolox
from torch.utils.data import DataLoader
from composer.datasets.coco_mmdet import mmdet_collate
from composer.core.data_spec import DataSpec
from composer.loggers import InMemoryLogger, LogLevel, WandBLogger



import logging, sys # disable logging in notebook
logging.disable(sys.maxsize)

torch.manual_seed(42) # For replicability

In [2]:
train_dataset = coco_mmdet(path='../../data/coco', split='train')
val_dataset = coco_mmdet(path='../../data/coco', split='val')

loading annotations into memory...
Done (t=11.58s)
creating index...
index created!
loading annotations into memory...
Done (t=0.36s)
creating index...
index created!


In [3]:
model = composer_yolox(model_name='yolox-s')

In [4]:
train_loader = DataLoader(train_dataset, batch_size=64, collate_fn=mmdet_collate, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, collate_fn=mmdet_collate, shuffle=False)

optimizer = dict(
    type='SGD',
    lr=0.01,
    momentum=0.9,
    weight_decay=5e-4,
    nesterov=True,
    paramwise_cfg=dict(norm_decay_mult=0., bias_decay_mult=0.))
optimizer_config = dict(grad_clip=None)

In [12]:
optimizer = composer.optim.DecoupledSGDW(
    model.parameters(), # Model parameters to update
    lr=0.01, # Peak learning rate
    momentum=0.9,
    weight_decay=5e-4,
    nesterov=True # If this looks large, it's because its not scaled by the LR as in non-decoupled weight decay
)

In [13]:
lr_scheduler = composer.optim.CosineAnnealingWithWarmupScheduler(
    t_warmup="30ep", # Warm up over 30 epoch
)

In [20]:
train_epochs = "300ep" # Train for 3 epochs because we're assuming Colab environment and hardware

trainer = composer.trainer.Trainer(
    model=model,
    train_dataloader=DataSpec(train_loader, get_num_samples_in_batch=lambda x: 64),
    eval_dataloader=DataSpec(val_loader, get_num_samples_in_batch=lambda x: 64),
    max_duration=train_epochs,
    optimizers=optimizer,
    schedulers=lr_scheduler,
    device="gpu" if torch.cuda.is_available() else "cpu",
    loggers=[InMemoryLogger(log_level=LogLevel.BATCH), WandBLogger(project='yolox-test')])


wandb: Currently logged in as: austin-j (mosaic-ml). Use `wandb login --relogin` to force relogin


In [21]:
trainer.fit()

Epoch     0 train   0%|                         | 0/9858 [00:00<?, ?it/s]         

In [9]:
trainer.state.current_metrics

{'eval': {'val_map': tensor(0.),
  'val_map_50': tensor(0.),
  'val_map_75': tensor(0.),
  'val_map_small': tensor(0.),
  'val_map_medium': tensor(0.),
  'val_map_large': tensor(0.),
  'val_mar_1': tensor(0.),
  'val_mar_10': tensor(0.),
  'val_mar_100': tensor(0.),
  'val_mar_small': tensor(0.),
  'val_mar_medium': tensor(0.),
  'val_mar_large': tensor(0.),
  'val_map_per_class': tensor(-1.),
  'val_mar_100_per_class': tensor(-1.)}}

In [ ]:
WandBLogger(project='yolox-test')